# Hypothetical Document Embedding (HyDE) in Document Retrieval

## Tổng quan

Đoạn mã này triển khai hệ thống Nhúng Tài liệu Giả định (HyDE) để truy xuất tài liệu. HyDE là một phương pháp tiếp cận sáng tạo, chuyển đổi các câu hỏi truy vấn thành các tài liệu giả định chứa câu trả lời, nhằm thu hẹp khoảng cách giữa phân phối truy vấn và tài liệu trong không gian vector.

## Động lực

Các phương pháp truy xuất truyền thống thường gặp khó khăn với khoảng cách ngữ nghĩa giữa các truy vấn ngắn và các tài liệu dài hơn, chi tiết hơn. HyDE giải quyết vấn đề này bằng cách mở rộng truy vấn thành một tài liệu giả định đầy đủ, có khả năng cải thiện mức độ liên quan của truy xuất bằng cách làm cho biểu diễn truy vấn tương tự hơn với biểu diễn tài liệu trong không gian vector.

## Các thành phần chính

1. Xử lý PDF và phân đoạn văn bản (text chunking)
2. Tạo kho vector sử dụng FAISS và nhúng OpenAI (OpenAI embeddings)
3. Mô hình ngôn ngữ để tạo tài liệu giả định
4. Lớp `HyDERetriever` tùy chỉnh triển khai kỹ thuật HyDE

## Chi tiết Phương pháp

### Tiền xử lý Tài liệu và Tạo Kho Vector

1. PDF được xử lý và chia thành các đoạn (chunks).
2. Một kho vector FAISS được tạo bằng cách sử dụng nhúng OpenAI để tìm kiếm độ tương tự hiệu quả.

### Tạo Tài liệu Giả định

1. Một mô hình ngôn ngữ (GPT-4) được sử dụng để tạo một tài liệu giả định trả lời truy vấn đã cho.
2. Việc tạo được hướng dẫn bởi một mẫu prompt đảm bảo tài liệu giả định chi tiết và phù hợp với kích thước đoạn (chunk size) được sử dụng trong kho vector.

### Quy trình Truy xuất

Lớp `HyDERetriever` triển khai các bước sau:

1. Tạo một tài liệu giả định từ truy vấn bằng mô hình ngôn ngữ.
2. Sử dụng tài liệu giả định làm truy vấn tìm kiếm trong kho vector.
3. Truy xuất các tài liệu tương tự nhất với tài liệu giả định này.

## Các Tính năng Chính

1. Mở rộng Truy vấn: Chuyển đổi các truy vấn ngắn thành các tài liệu giả định chi tiết.
2. Cấu hình Linh hoạt: Cho phép điều chỉnh kích thước đoạn (chunk size), độ chồng chéo (overlap) và số lượng tài liệu được truy xuất.
3. Tích hợp với Mô hình OpenAI: Sử dụng GPT-4 để tạo tài liệu giả định và nhúng OpenAI để biểu diễn vector.

## Lợi ích của Phương pháp này

1. Cải thiện Mức độ Liên quan: Bằng cách mở rộng truy vấn thành các tài liệu đầy đủ, HyDE có khả năng nắm bắt các kết quả phù hợp chi tiết và liên quan hơn.
2. Xử lý các Truy vấn Phức tạp: Đặc biệt hữu ích cho các truy vấn phức tạp hoặc đa khía cạnh có thể khó khớp trực tiếp.
3. Khả năng Thích ứng: Việc tạo tài liệu giả định có thể thích ứng với các loại truy vấn và miền tài liệu khác nhau.
4. Tiềm năng Hiểu Ngữ cảnh Tốt hơn: Truy vấn mở rộng có thể nắm bắt ngữ cảnh và ý định đằng sau câu hỏi gốc tốt hơn.

## Chi tiết Triển khai

1. Sử dụng mô hình ChatGPT của OpenAI để tạo tài liệu giả định.
2. Sử dụng FAISS để tìm kiếm độ tương tự hiệu quả trong không gian vector.
3. Cho phép dễ dàng trực quan hóa cả tài liệu giả định và kết quả được truy xuất.

## Kết luận

Nhúng Tài liệu Giả định (HyDE) đại diện cho một phương pháp tiếp cận sáng tạo để truy xuất tài liệu, giải quyết khoảng cách ngữ nghĩa giữa truy vấn và tài liệu. Bằng cách tận dụng các mô hình ngôn ngữ tiên tiến để mở rộng truy vấn thành các tài liệu giả định, HyDE có khả năng cải thiện đáng kể mức độ liên quan của truy xuất, đặc biệt đối với các truy vấn phức tạp hoặc chi tiết. Kỹ thuật này có thể đặc biệt có giá trị trong các miền mà việc hiểu ý định và ngữ cảnh truy vấn là rất quan trọng, chẳng hạn như nghiên cứu pháp lý, đánh giá tài liệu học thuật hoặc hệ thống truy xuất thông tin nâng cao.

<div style="text-align: center;">

<img src="images/HyDe.svg" alt="HyDe" style="width:40%; height:auto;">
</div>


<div style="text-align: center;">

<img src="images/hyde-advantages.svg" alt="HyDe" style="width:100%; height:auto;">
</div>

In [ ]:
# Basic setup
import os
from langchain_ollama import ChatOllama
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma

llm_model_name='qwen2.5:7b'
llm = ChatOllama(model=llm_model_name, temperature=0)

embedding_model='bge-m3:latest'
embeddings = OllamaEmbeddings(model=embedding_model)

vectorstore = Chroma(
    collection_name="GTDB_35_v3",
    embedding_function=embeddings,
    persist_directory="data/chroma_GTDB_35_db", 
)

retriever = vectorstore.as_retriever(
  search_type="similarity",
  search_kwargs={'k': 4}, # number of documents to retrieve
)

In [3]:
# Test
question = "Quy định đặt tên đường bộ là gì?"
retriever_docs = retriever.invoke(question)
retriever_docs

[Document(id='ec27fa39-ec2a-4a16-a8fd-60bf32bf7d57', metadata={'creationdate': '2024-09-20T10:02:58+07:00', 'creator': 'PyPDF', 'moddate': '2024-09-20T10:02:58+07:00', 'page': 7, 'page_label': '8', 'producer': 'PyPDF', 'source': 'law_gtdb/data/law_gtdb_35.pdf', 'total_pages': 69}, page_content='Đặt tên, đổi tên, số hiệu đường bộ \n1. Việc đặt tên, số hiệu đường bộ được quy định như sau: \na) Tên đường bộ được đặt theo tên danh nhân, người có công với đất nước; di \ntích, sự kiện lịch sử, văn hóa; tên địa danh hoặc tên theo tập quán. Số hiệu đường \nbộ được đặt theo số tự nhiên hoặc số tự nhiên kèm theo chữ cái nếu cần thiết. Trường hợp đường đô thị trùng với đường khác thì sử dụng cả tên đường đô thị và \ntên, số hiệu của đường khác; \nb) Tên, số hiệu đường bộ tham gia vào mạng lưới đường bộ quốc tế thực hiện \ntheo điều ước quốc tế giữa nước Cộng hòa xã hội chủ nghĩa Việt Nam với các quốc \ngia, tổ chức quốc tế có liên quan. Đường bộ kết nối vào mạng lưới đường bộ quốc tế \nthì sử dụn

### Define the HyDe retriever class - creating vector store, generating hypothetical document, and retrieving

In [4]:
from langchain_core.prompts import PromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

def replace_t_with_space(list_of_documents):
    """
    Thay thế tất cả các ký tự tab ('\t') bằng dấu cách trong nội dung trang của mỗi tài liệu

    Args:
        list_of_documents: Danh sách các đối tượng tài liệu, mỗi đối tượng có thuộc tính 'page_content'.

    Returns:
        Danh sách tài liệu đã được sửa đổi với các ký tự tab được thay thế bằng dấu cách.
    """

    for doc in list_of_documents:
        doc.page_content = doc.page_content.replace('\t', ' ')  # Replace tabs with spaces
    return list_of_documents

class HyDERetriever:
    def __init__(self, files_path, chunk_size=500, chunk_overlap=100):
        llm_model_name='qwen2.5:7b'
        self.llm = ChatOllama(model=llm_model_name, temperature=0)

        embedding_model='bge-m3:latest'
        self.embeddings = OllamaEmbeddings(model=embedding_model)
        
        self.chunk_size = chunk_size
        self.chunk_overlab = chunk_overlap
        
        # Load PDF documents
        loader = PyPDFLoader(files_path)
        documents = loader.load()

        # Split documents into chunks
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len
        )
        texts = text_splitter.split_documents(documents)
        cleaned_texts = replace_t_with_space(texts)

        # Create embeddings and vector store
        self.vectorstore = Chroma.from_documents(
            documents=cleaned_texts,
            collection_name='GTDB_35_v4',
            embedding=self.embeddings)
        
        self.hyde_prompt = PromptTemplate(
            input_variables=["query", "chunk_size"],
            template="""
            Given the question '{query}', generate a hypothetical document that directly answers this question. 
            The document should be detailed and in-depth.
            The document size has be exactly {chunk_size} characters.""",
        )
        self.hyde_chain = self.hyde_prompt | self.llm

    def generate_hypothetical_document(self, query):
        input_variables = {"query": query, "chunk_size": self.chunk_size}
        return self.hyde_chain.invoke(input_variables).content

    def retrieve(self, query, k=3):
        hypothetical_doc = self.generate_hypothetical_document(query)
        similar_docs = self.vectorstore.similarity_search(hypothetical_doc, k=k)
        return similar_docs, hypothetical_doc


In [5]:
PDF_FILE_PATH='law_gtdb/data/law_gtdb_35.pdf'
retriever = HyDERetriever(PDF_FILE_PATH)

In [6]:
question = "Quy định đặt tên đường bộ là gì?"
results, hypothetical_doc = retriever.retrieve(question)

In [7]:
import textwrap

def show_context(context):
    """
    Hiển thị nội dung của danh sách ngữ cảnh được cung cấp.

    Args:
        context (list): Một danh sách các mục ngữ cảnh cần hiển thị.

    In ra từng mục ngữ cảnh trong danh sách với tiêu đề chỉ vị trí của nó.
    """
    for i, c in enumerate(context):
        print(f"Context {i + 1}:")
        print(c)
        print("\n")

def text_wrap(text, width=120):
    """
    Bao bọc văn bản đầu vào theo chiều rộng được chỉ định.

    Args:
        text (str): Văn bản đầu vào cần bao bọc.
        width (int): Chiều rộng để bao bọc văn bản.

    Returns:
        str: Văn bản đã được bao bọc.
    """
    return textwrap.fill(text, width=width)

In [8]:
docs_content = [doc.page_content for doc in results]

print("hypothetical_doc:\n")
print(text_wrap(hypothetical_doc)+"\n")
show_context(docs_content)

hypothetical_doc:

Quy định đặt tên đường bộ tại Việt Nam được quy định chi tiết trong Nghị định số 13/2020/NĐ-CP của Chính phủ, có hiệu
lực từ ngày 01/04/2020. Theo đó, việc đặt tên đường bộ phải tuân thủ các nguyên tắc: Tên đường phải ngắn gọn, dễ nhớ và
phù hợp với đặc điểm địa lý, lịch sử, văn hóa của khu vực; không được trùng lặp với tên đường khác trong cùng một thành
phố hoặc thị xã; không sử dụng từ ngữ mang tính chính trị, tôn giáo, thương mại. Đồng thời, quy định này cũng yêu cầu
các cơ quan có thẩm quyền phải công khai, lấy ý kiến góp ý từ cộng đồng trước khi quyết định đặt tên mới cho đường bộ.

Context 1:
10 CÔNG BÁO/Số 983 + 984/Ngày 25-8-2024 
  
4. Cấp kỹ thuật của đường bộ quy định tại điểm c khoản 2 Điều này thực hiện 
theo quy chuẩn kỹ thuật quốc gia do Bộ trưởng Bộ Xây dựng ban hành và tiêu 
chuẩn kỹ thuật do cấp có thẩm quyền công bố. 
Điều 11. Đặt tên, đổi tên, số hiệu đường bộ 
1. Việc đặt tên, số hiệu đường bộ được quy định như sau: 
a) Tên đường bộ được đặt the